# Verse Lines with Metrical Analysis

## Loading the Initial Data

In [1]:
import pandas

# see 4_verse_lines.ipynb for details on this file
df_rv_lines = pandas.read_csv("data/rv_lines.csv", keep_default_na=False)

#df_rv_lines.head()

In [2]:
# some tests

#df_rv_lines.dtypes

# filtering only on a specific meter
#df_rv_lines.query('stanza_meter == "Jagatī"').sort_values('line_no')

## Adding Metrical Analysis Data

Adding data on syllabification, scansion, meter correctness and stem position for each of our verse lines

In [3]:
# in the same folder as this notebook
import src.lib.meter as meter

# useful during testing to pick up changes in the file
import importlib
importlib.reload(meter)


def add_meter_analysis(line):
    analysis = meter.analyze(line["text_samhitapatha"], line["stanza_meter"], line["stem"])

    # disabling the guess of stanza meter, does not alter our core result (count of nA/nI in
    # expected short syllables) even with a very permissive guess so no point in pursuing
    # this avenue right now, since we need to be careful with the guess assignment anyway
    #if "stanza_meter_guessed" in analysis:
    #    # TODO add guessed at the end here?
    #    line["stanza_meter"] = analysis["stanza_meter_guessed"]
    
    # FIXME check if avagraha (o_' / o_a) is present in our lines of interest and
    # decide how to deal with the 'o' vowel    
    #if "o a" in line["text_samhitapatha"]:
    #    print(f'{line["line_no"]}: {line["text_samhitapatha"]}')

    # TODO rename this as the default meter scansion
    # scansion results
    line["meter_scansion_custom"] = analysis["scansion"]
    line["meter_has_restorations"] = analysis["has_restorations"]
    line["meter_caesura_position"] = analysis["caesura_position"]
    line["meter_syllables_count"] = analysis["no_of_syllables"]
    line["meter_syllables"] = analysis["syllables"]
    line["meter_notes"] = analysis["notes"]
    line["meter_scansions_match"] = \
        analysis["scansion_syllables"] == meter.clean_meter_scansion(line["meter_scansion"])
    
    # we check for these in the next section by querying dataframe directly
    #if not line["meter_scansions_match"]:
    #    print(
    #        f"{line['line_no']}:", 
    #        f"{line['meter_scansion_custom']} ≠ {line['meter_scansion']}",
    #        f"[{line['text_samhitapatha']}] ({line['meter_syllables_count']} {line['stanza_meter']})"
    #    )
    
    # fields related to meter correctness
    line["meter_is_correct"] = analysis["is_correct"]
    line["meter_faults"] = analysis["faults"]
    line["meter_fault_positions"] = analysis["fault_positions"]
    
    # text that reflects the final metrical analysis really
    line["text_samhitapatha_normalized"] = analysis["text_normalized"]

    # where in the meter the stem is and which variant of the stem was found
    line["meter_stem_found"] = analysis["search_term_found"]
    line["meter_stem_positions"] = analysis["search_term_positions"]
    line["meter_stem_fault_positions"] = analysis["search_term_fault_positions"]
    
    # compute extra positions needed for our analysis
    # TODO! check for multiple matches in one line 
    line["meter_stem_vowel_position"] = line["meter_stem_positions"][-1] # last item
    line["meter_stem_root_vowel_position"] = line["meter_stem_positions"][:-1][-1] # last item after ignoring stem vowel
    # TODO remove this?
    meter_stem_fault_position_labels = []
    for position in line["meter_stem_fault_positions"]:
        if position == line["meter_stem_vowel_position"]:
            label = "stem_vowel"
        elif position == line["meter_stem_root_vowel_position"]:
            label = "root_vowel"
        else:
            label = "preceding_syllable"
        meter_stem_fault_position_labels.append(label)
    line["meter_stem_fault_position_labels"] = " ".join(meter_stem_fault_position_labels)
    
    # add expected scansion for the stem vowel and root vowel
    line["meter_stem_vowel_position_scansion_expected"] = meter.get_expected_scansion(
        line["meter_stem_vowel_position"], line["stanza_meter"], line["meter_caesura_position"]
    )
    line["meter_stem_root_vowel_position_scansion_expected"] = meter.get_expected_scansion(
        line["meter_stem_root_vowel_position"], line["stanza_meter"], line["meter_caesura_position"]
    ) 
        
    return line

df_rv_lines_with_meter = df_rv_lines \
    .apply(add_meter_analysis, axis=1) # axis=1 makes this operate on rows

df_rv_lines_with_meter.to_dict("records")[0] # sample

{'line_no': '01.063.02.d',
 'stem': 'iṣṇā',
 'stem_type': 'strong',
 'root_guess': 'iṣ',
 'variant_no': '',
 'weak_only': False,
 'attestation_texts': '',
 'language_period': 'Earlier',
 'present_class': 'ninth',
 'root': 'iṣ 1',
 'book': 1,
 'hymn': 1.063,
 'stanza': '01.063.02',
 'pada_id': 'd',
 'pada_label': 'M',
 'word': 'iṣṇā́si',
 'word_position_no': 2,
 'word_position': 'intermediate',
 'word_gloss': '2.SG.PRS.IND.ACT',
 'text_padapatha': 'púraḥ iṣṇā́si= puruhūta pūrvī́ḥ',
 'text_samhitapatha': 'púra iṣṇā́si puruhūta pūrvī́ḥ',
 'meter_scansion': 'SS LLS SSLS LL',
 'stanza_meter': 'Triṣṭubh',
 'stanza_strata': 'A',
 'stanza_late_addition': '',
 'hymn_absolute_no': 63,
 'hymn_addressee': 'Indra',
 'hymn_group': 'Hymns of Nodhas, Descendant of Gotama',
 'meter_scansion_custom': 'SS LLS ,SS|LS LL',
 'meter_has_restorations': False,
 'meter_caesura_position': 5,
 'meter_syllables_count': 11,
 'meter_syllables': ['pú',
  'ra',
  'iṣ',
  'ṇā́',
  'si',
  'pu',
  'ru',
  'hū',
  'ta'

## Saving the Results

In [4]:
df_rv_lines_with_meter.to_csv("data/rv_lines_with_meter.csv", index=None)

df_rv_lines_with_meter.head()

,line_no,stem,stem_type,root_guess,variant_no,weak_only,attestation_texts,language_period,present_class,root,...,meter_fault_positions,text_samhitapatha_normalized,meter_stem_found,meter_stem_positions,meter_stem_fault_positions,meter_stem_vowel_position,meter_stem_root_vowel_position,meter_stem_fault_position_labels,meter_stem_vowel_position_scansion_expected,meter_stem_root_vowel_position_scansion_expected
0,01.063.02.d,iṣṇā,strong,iṣ,,False,,Earlier,ninth,iṣ 1,...,[],púra iṣṇā́si puruhūta pūrvī́ḥ,iṣṇā,"[3, 4]",[],4,3,,X,X
1,01.063.04.b,ubhnā,strong,ubh,,False,V.,Earlier,ninth,ubh,...,[],vr̥tráṁ yád vajrin vr̥ṣakarman ubhnā́ḥ,ubhnā,"[10, 11]",[],11,10,,X,L
2,04.019.04.c,ubhnā,strong,ubh,,False,V.,Earlier,ninth,ubh,...,[],dr̥̄ḷhā́ni aubhnād uśámāna ójo,ubhnā,"[4, 5]",[],5,4,,,
3,04.018.12.d,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,ninth,kṣī,...,[],yát prā́kṣiṇāḥ pitáram pādagŕ̥hya,kṣiṇā,"[2, 3, 4]",[],4,3,,X,X
4,10.027.04.d,kṣiṇā,strong,kṣi,,False,V.B.,Earlier,ninth,kṣī,...,[],prá táṁ kṣiṇām párvate pādagŕ̥hya,kṣiṇā,"[3, 4]",[],4,3,,X,X


## Validation

### Running tests for our custom meter scansion

In [5]:
!python src/test_meter_analysis.py

agním īḷe puróhitaṁ | Gāyatrī
parts: ['ag', 'ní', 'm ī', 'ḷe', ' ', 'pu', 'ró', 'hi', 'taṁ']
scansion: LS_LL |SLSL
caesura_position: -1

yajñásya devám r̥tvíjam | Gāyatrī
parts: ['yaj', 'ñás', 'ya', ' ', 'de', 'vá', 'm r̥t', 'ví', 'jam']
scansion: LLS L|S_LSL
caesura_position: -1

hótāraṁ  ratnadhā́tamam  | Gāyatrī
parts: ['hó', 'tā', 'raṁ', ' ', 'rat', 'na', 'dhā́', 'ta', 'mam']
scansion: LLL L|SLSL
caesura_position: -1

sá íd devéṣu gachati | Gāyatrī
parts: ['sá', ' ', 'íd', ' ', 'de', 'vé', 'ṣu', ' ', 'gac', 'cha', 'ti']
scansion: S L LL|S LSS
caesura_position: -1

víśve devā́so aptúraḥ | Gāyatrī
parts: ['víś', 've', ' ', 'de', 'vā́', 'so', ' ', 'ap', 'tú', 'raḥ']
scansion: LL LL|L LSL
caesura_position: -1
Faults: scansion_cadence=LLSX

ví tigména vr̥ṣabhéṇā púro 'bhet | Triṣṭubh
parts: ['ví', ' ', 'tig', 'mé', 'na', ' ', 'vr̥', 'ṣa', 'bhé', 'ṇā', ' ', 'pú', 'ro', ' ', "'bhet"]
scansion: S LLS ,SSL|L SL L
caesura_position: 4

áditer dákṣo 'jāyata@ | Aṇuṣṭubh
parts: ['á', 'di', 

### Checking our scansion yields same final result as vedaweb

In [6]:
COLUMNS_SELECTED = [
    "present_class", "stem", "line_no", "text_padapatha",
    "text_samhitapatha", "text_samhitapatha_normalized",
    "meter_scansion", "meter_scansion_custom",
    "meter_syllables", "meter_syllables_count",
    "stanza_meter",
    "meter_caesura_position",
    "meter_has_restorations", "meter_notes",
    "meter_scansions_match",
    "meter_is_correct", "meter_faults",
    "meter_stem_found", "meter_stem_positions",
    "meter_stem_root_vowel_position", "meter_stem_vowel_position",
    "meter_stem_root_vowel_position_scansion_expected", "meter_stem_vowel_position_scansion_expected", 
    "meter_fault_positions",
    "meter_stem_fault_positions", "meter_stem_fault_position_labels",
]

df_rv_lines_with_meter[COLUMNS_SELECTED].query('meter_scansions_match == False')

,present_class,stem,line_no,text_padapatha,text_samhitapatha,text_samhitapatha_normalized,meter_scansion,meter_scansion_custom,meter_syllables,meter_syllables_count,...,meter_faults,meter_stem_found,meter_stem_positions,meter_stem_root_vowel_position,meter_stem_vowel_position,meter_stem_root_vowel_position_scansion_expected,meter_stem_vowel_position_scansion_expected,meter_fault_positions,meter_stem_fault_positions,meter_stem_fault_position_labels
2,ninth,ubhnā,04.019.04.c,dr̥ḷhā́ni aubhnāt= uśámānaḥ ójaḥ,dr̥r̥ḷhā́ni+ aubhnād uśámāna ójo,dr̥̄ḷhā́ni aubhnād uśámāna ójo,SLLS LL SSLS LL,LLS LL_SSLS LL,"[dr̥̄, ḷhā́, ni, aubh, nā, d u, śá, mā, na, ó,...",11,...,,ubhnā,"[4, 5]",4,5,,,[],[],
58,ninth,minā mīnā,03.049.02.d,pr̥thujráyāḥ= amināt ā́yuḥ dásyoḥ,pr̥thujráyā aminād ā́yu@ dásyoḥ,pr̥thujráyā aminād ā́yu dásyoḥ,SLSL SSL LL LL,"SLSL ,SSL|_LS LL","[pr̥, thuj, rá, yā, a, mi, nā, d ā́, yu, dás, ...",11,...,,minā,"[6, 7]",6,7,S,X,[],[],
111,ninth,śrīṇī,09.011.06.b,dadhnā́ ít abhí śrīṇītana,dadhnéd abhí śriṇītana@,dadhnéd abhí śriṇītana,LL SL SLSL,LL_S_L|SLSS,"[dadh, né, d a, bhí ś, ri, ṇī, ta, na]",8,...,,śriṇī,"[4, 5, 6]",5,6,S,L,[],[],
167,ninth,gr̥ṇā,01.048.04.d,nā́ma gr̥ṇāti nr̥ṇā́m,nā́ma gr̥ṇāti nr̥r̥ṇáam+,nā́ma gr̥ṇāti nr̥̄ṇáam,LS SLS SSSL,LS SLS LSL,"[nā́, ma, gr̥, ṇā, ti, nr̥̄, ṇá, am]",8,...,,gr̥ṇā,"[3, 4]",3,4,,,[],[],
363,fifth,r̥ṇo,01.174.09.b,r̥ṇóḥ apáḥ sīrā́ḥ ná+_ srávantīḥ,r̥ṇór apáḥ ̀ sīrā́ ná srávantīḥ,r̥ṇór apáḥ ̀ sīrā́ ná srávantīḥ,SL SS · LL L SLL,"SL_SL · ,LL |_LSLL","[r̥, ṇó, r a, páḥ, ̀, sī, rā́, ná s, rá, van,...",11,...,scansion_post_caesura=LL,r̥ṇo,"[1, 2]",1,2,X,X,[7],[],
365,fifth,r̥ṇo,06.020.12.b,r̥ṇóḥ apáḥ sīrā́ḥ ná+_ srávantīḥ,r̥ṇór apáḥ ̀ sīrā́ ná srávantīḥ,r̥ṇór apáḥ ̀ sīrā́ ná srávantīḥ,SL SS · LL L SLL,"SL_SL · ,LL |_LSLL","[r̥, ṇó, r a, páḥ, ̀, sī, rā́, ná s, rá, van,...",11,...,scansion_post_caesura=LL,r̥ṇo,"[1, 2]",1,2,X,X,[7],[],
493,fifth,kr̥ṇu,10.034.14.a,mitrám kr̥ṇudhvam= khálu mr̥ḷátā+ naḥ,mitráṁ kr̥ṇudhvaṁ khálu mr̥r̥ḷátā+ no,mitráṁ kr̥ṇudhvaṁ khálu mr̥̄ḷátā no,LL SLL SS SSSL L,LL SLL SS LSL L,"[mit, ráṁ, kr̥, ṇudh, vaṁ, khá, lu, mr̥̄, ḷá, ...",11,...,,kr̥ṇu,"[3, 4]",3,4,,,[],[],
681,fifth,dhunu dhūnu,01.010.08.d,sám gā́ḥ asmábhyam dhūnuhi,sáṁ gā́ asmábhya@ dhūnuhi,sáṁ gā́ asmábhya dhūnuhi,L L LLL LSS,L L LL|S LSS,"[sáṁ, gā́, as, mábh, ya, dhū, nu, hi]",8,...,,dhūnu,"[6, 7]",6,7,L,S,[],[],
769,fifth,śr̥ṇu,08.084.03.b,nr̥ŕ̥n pāhi śr̥ṇudhī́?_+ gíraḥ,nr̥ŕ̥m̐ḥ pāhi śr̥ṇudhī́ gíraḥ,nr̥̄́m̐ḥ pāhi śr̥ṇudhī́ gíraḥ,SL LS SSL SL,L_LS SSL SL,"[nr̥̄́m̐, ḥ pā, hi, śr̥, ṇu, dhī́, gí, raḥ]",8,...,,śr̥ṇu,"[4, 5]",4,5,,,[],[],


These reasons for differences here can be safely ignored:

* Vedaweb does not clean the character `@` (used to mark [edited forms](https://lrc.la.utexas.edu/books/rigveda/RV00#bolle)) from the vnh text before applying metrical scansion while we do.
* Vedaweb currently treats `ḷh` as combination of two characters while we correctly treat it as single.
* Vedaweb currently treats `ḷ` between consonants as a consonant too, while we correctly treat it as the vowel `l̥`. Note: instances of `ḷ` in our lines are all consonantal (as in 04.019.04.c, 07.007.03.b, 10.089.06.d, 02.013.10.c), hence we don't actually see this in action in the above table.
* Vedaweb treats sequences `r̥r̥` and `r̥ŕ̥` as different chars, while we correctly treat them as single unit r̥̄ and r̥̄́ respectively (`ŕ̥r̥` is still treated as different chars which is needed for the meter). As in: 04.019.04.c, 01.048.04.d (as seen in the above table too)
* Vedaweb treats sequence like `paḥ` as short before a metrical pause, while we correctly treat them as long.

Note: [04.019.04.c](https://vedaweb.uni-koeln.de/rigveda/view/id/4.19.4) doesn't have a meter specified but other stanzas in the hymn are Triṣṭubh, and once we treat `r̥r̥` as a unit, it actually follows the meter.

#### Text normalization notes

`l̥` and `r̥̄` are not actually not to be found in the vnh text we have, since it's transliterated using IAST, which [merges](https://en.wikipedia.org/wiki/International_Alphabet_of_Sanskrit_Transliteration#Comparison_with_ISO_15919) vowel `l̥` with consonant `ḷ` and for `r̥̄`, writes `r̥r̥` instead. This has consequences for the meter, so we normalize the instances of those before we do the metrical analysis.

Examples demonstrating this behaviour in vedaweb:

https://vedaweb.uni-koeln.de/rigveda/view/id/10.157.02  
https://vedaweb.uni-koeln.de/rigveda/view/id/01.030.15

In the texas version, we don't see these issues:

https://lrc.la.utexas.edu/books/rigveda/RV10#H157  
https://lrc.la.utexas.edu/books/rigveda/RV01#H030

Similarly, we normalize `ch` spelling to `cch`.

### Checking no of lines that have had metrical restorations

Lines that have metrical restorations done ([e.g.](https://vedaweb.uni-koeln.de/rigveda/view/id/9.11.6)) For more details, see https://lrc.la.utexas.edu/books/rigveda/RV00#bolle 

In [7]:
df_rv_lines_with_meter[COLUMNS_SELECTED].query('meter_has_restorations == True')

,present_class,stem,line_no,text_padapatha,text_samhitapatha,text_samhitapatha_normalized,meter_scansion,meter_scansion_custom,meter_syllables,meter_syllables_count,...,meter_faults,meter_stem_found,meter_stem_positions,meter_stem_root_vowel_position,meter_stem_vowel_position,meter_stem_root_vowel_position_scansion_expected,meter_stem_vowel_position_scansion_expected,meter_fault_positions,meter_stem_fault_positions,meter_stem_fault_position_labels
2,ninth,ubhnā,04.019.04.c,dr̥ḷhā́ni aubhnāt= uśámānaḥ ójaḥ,dr̥r̥ḷhā́ni+ aubhnād uśámāna ójo,dr̥̄ḷhā́ni aubhnād uśámāna ójo,SLLS LL SSLS LL,LLS LL_SSLS LL,"[dr̥̄, ḷhā́, ni, aubh, nā, d u, śá, mā, na, ó,...",11,...,,ubhnā,"[4, 5]",4,5,,,[],[],
31,ninth,pr̥ṇī,05.005.05.c,prá-pra yajñám pr̥ṇītana,prá-pra yajñám pr̥ṇītana,prápra yajñám pr̥ṇītana,LS LL SLSS,LS LL |SLSS,"[práp, ra, yaj, ñám, pr̥, ṇī, ta, na]",8,...,,pr̥ṇī,"[5, 6]",5,6,S,L,[],[],
48,ninth,minī mīnī,01.025.01.c,minīmási dyávi-dyavi,minīmási dyávi-dyavi,minīmási dyávidyavi,SLSL SLSS,SLS_L|SLSS,"[mi, nī, má, si d, yá, vid, ya, vi]",8,...,,minī,"[1, 2]",1,2,X,X,[],[],
55,ninth,minā mīnā,02.012.05.c,sáḥ aryáḥ puṣṭī́ḥ= víjaḥ iva ā́} mināti,só aryáḥ puṣṭī́r víja 'vā́@ mināti,só aryáḥ puṣṭī́r víja 'vā́ mināti,L LL LL SS L SLS,"L LL LL ,SS |L SLS","[só, ar, yáḥ, puṣ, ṭī́r, ví, ja, 'vā́, mi, nā,...",11,...,,minā,"[9, 10]",9,10,S,L,[],[],
58,ninth,minā mīnā,03.049.02.d,pr̥thujráyāḥ= amināt ā́yuḥ dásyoḥ,pr̥thujráyā aminād ā́yu@ dásyoḥ,pr̥thujráyā aminād ā́yu dásyoḥ,SLSL SSL LL LL,"SLSL ,SSL|_LS LL","[pr̥, thuj, rá, yā, a, mi, nā, d ā́, yu, dás, ...",11,...,,minā,"[6, 7]",6,7,S,X,[],[],
111,ninth,śrīṇī,09.011.06.b,dadhnā́ ít abhí śrīṇītana,dadhnéd abhí śriṇītana@,dadhnéd abhí śriṇītana,LL SL SLSL,LL_S_L|SLSS,"[dadh, né, d a, bhí ś, ri, ṇī, ta, na]",8,...,,śriṇī,"[4, 5, 6]",5,6,S,L,[],[],
123,ninth,aśnā,09.067.31.c,sárvam sá pūtám aśnāti,*sárvaṁ sá pūtám aśnāti,sárvaṁ sá pūtám aśnāti,LL S LS LLS,LL S L|S_LLS,"[sár, vaṁ, sá, pū, tá, m aś, nā, ti]",8,...,scansion_cadence=SLLX,aśnā,"[6, 7]",6,7,L,S,[7],[7],stem_vowel
135,ninth,gr̥ṇī,01.155.04.a,tát-tat ít asya= paúṁsyam} gr̥ṇīmasi,tát-tad íd asya paúṁsiyaṁ gr̥ṇīmasi,táttad íd asya paúṁsiyaṁ gr̥ṇīmasi,LS S LS LSL SLSS,LS_S_LS LSL SLSS,"[tát, ta, d í, d as, ya, paúṁ, si, yaṁ, gr̥, ṇ...",12,...,,gr̥ṇī,"[9, 10]",9,10,,,[],[],
136,ninth,gr̥ṇī,01.186.03.a,préṣṭham vaḥ átithim gr̥ṇīṣe-_,práyiṣṭhaṁ+ vo ̀ átithiṁ gr̥ṇīṣe,práyiṣṭhaṁ vo ̀ átithiṁ gr̥ṇīṣe,SLL L · SSL SLL,"SLL L · ,SS|L SLL","[prá, yiṣ, ṭhaṁ, vo, ̀, á, ti, thiṁ, gr̥, ṇī,...",11,...,,gr̥ṇī,"[9, 10]",9,10,S,L,[],[],
138,ninth,gr̥ṇī,02.020.04.a,tám u+_ stuṣe-_= índram tám} gr̥ṇīṣe-_,tám u stuṣa índaraṁ+ táṁ gr̥ṇīṣe,tám u stuṣa índaraṁ táṁ gr̥ṇīṣe,S L SS LSL L SLL,"S_LSS ,LSL |L SLL","[tá, m u s, tu, ṣa, ín, da, raṁ, táṁ, gr̥, ṇī,...",11,...,,gr̥ṇī,"[9, 10]",9,10,S,L,[],[],


Need to be mindful of these lines during our analysis.

### Checking no of lines that have other caesura positions eligible

In [8]:
df_rv_lines_with_meter[COLUMNS_SELECTED].query(
    'meter_notes.str.contains("caesura_position_alt")', engine='python'
)

,present_class,stem,line_no,text_padapatha,text_samhitapatha,text_samhitapatha_normalized,meter_scansion,meter_scansion_custom,meter_syllables,meter_syllables_count,...,meter_faults,meter_stem_found,meter_stem_positions,meter_stem_root_vowel_position,meter_stem_vowel_position,meter_stem_root_vowel_position_scansion_expected,meter_stem_vowel_position_scansion_expected,meter_fault_positions,meter_stem_fault_positions,meter_stem_fault_position_labels
36,ninth,pr̥ṇā,01.125.05.b,yáḥ pr̥ṇā́ti= sá ha devéṣu gacchati,yáḥ pr̥ṇā́ti sá ha devéṣu gachati,yáḥ pr̥ṇā́ti sá ha devéṣu gacchati,L SLS S S LLS LSS,"L SLS ,S S L|LS LSS","[yáḥ, pr̥, ṇā́, ti, sá, ha, de, vé, ṣu, gac, c...",12,...,,pr̥ṇā,"[2, 3]",2,3,X,X,[],[],
37,ninth,pr̥ṇā,02.030.07.c,yáḥ me-_ pr̥ṇā́t= yáḥ dádat yáḥ} nibódhāt,yó me pr̥ṇā́d yó dádad yó nibódhād,yó me pr̥ṇā́d yó dádad yó nibódhād,L L SL L SL L SLL,"L L SL ,L SL |L SLL","[yó, me, pr̥, ṇā́d, yó, dá, dad, yó, ni, bó, d...",11,...,,pr̥ṇā,"[3, 4]",3,4,X,X,[],[],
40,ninth,pr̥ṇā,06.047.15.a,káḥ īm stavat= káḥ pr̥ṇāt káḥ} yajāte?_,ká īṁ stavat káḥ pr̥ṇāt kó yajāte,ká īṁ stavat káḥ pr̥ṇāt kó yajāte,S L SL L SL L SLL,"S L SL ,L SL |L SLL","[ká, īṁ, sta, vat, káḥ, pr̥, ṇāt, kó, ya, jā, te]",11,...,,pr̥ṇā,"[6, 7]",6,7,S,X,[],[],
56,ninth,minā mīnā,03.030.12.a,díśaḥ sū́ryaḥ= ná mināti} prádiṣṭāḥ,díśaḥ sū́ryo ná mināti prádiṣṭā,díśaḥ sū́ryo ná mināti prádiṣṭā,SL LL S SLL SLL,"SL LL ,S SL|_LSLL","[dí, śaḥ, sū́r, yo, ná, mi, nā, ti p, rá, diṣ,...",11,...,,minā,"[6, 7]",6,7,S,X,[],[],
117,ninth,hr̥ṇī,02.033.15.b,yáthā deva= ná hr̥ṇīṣé?_} ná háṁsi,yáthā deva ná hr̥ṇīṣé ná háṁsi,yáthā deva ná hr̥ṇīṣé ná háṁsi,SL LS S SLL S LS,"SL LS ,S SL|L S LS","[yá, thā, de, va, ná, hr̥, ṇī, ṣé, ná, háṁ, si]",11,...,,hr̥ṇī,"[6, 7]",6,7,S,X,[],[],
188,ninth,jānā,06.009.02.a,ná ahám tántum= ná ví jānāmi ótum,nā́háṁ tántuṁ ná ví jānāmi ótuṁ,nā́háṁ tántuṁ ná ví jānāmi ótuṁ,LL LL S S LLS LL,"LL LL ,S S L|LS LL","[nā́, háṁ, tán, tuṁ, ná, ví, jā, nā, mi, ó, tuṁ]",11,...,,jānā,"[7, 8]",7,8,X,L,[],[],
189,ninth,jānā,06.009.03.a,sáḥ ít tántum= sá ví jānāti ótum,sá ít tántuṁ sá ví jānāti ótuṁ,sá ít tántuṁ sá ví jānāti ótuṁ,S L LL S S LLS LL,"S L LL ,S S L|LS LL","[sá, ít, tán, tuṁ, sá, ví, jā, nā, ti, ó, tuṁ]",11,...,,jānā,"[7, 8]",7,8,X,L,[],[],
364,fifth,r̥ṇo,06.018.05.d,r̥ṇóḥ púraḥ= ví dúraḥ asya víśvāḥ,r̥ṇóḥ púro ví dúro asya víśvāḥ,r̥ṇóḥ púro ví dúro asya víśvāḥ,SL SL S SL LS LL,"SL SL ,S SL |LS LL","[r̥, ṇóḥ, pú, ro, ví, dú, ro, as, ya, víś, vāḥ]",11,...,,r̥ṇo,"[1, 2]",1,2,X,X,[],[],
563,fifth,kr̥ṇo,04.050.09.c,avasyáve?_= yáḥ várivaḥ} kr̥ṇóti,avasyáve yó várivaḥ kr̥ṇóti,avasyáve yó várivaḥ kr̥ṇóti,SLSL L SSL SLS,"SLSL ,L SS|L SLS","[a, vas, yá, ve, yó, vá, ri, vaḥ, kr̥, ṇó, ti]",11,...,,kr̥ṇo,"[9, 10]",9,10,S,L,[],[],
634,fifth,dabhnu,01.055.07.d,ná tvā kétāḥ= ā́ dabhnuvanti bhū́rṇayaḥ,ná tvā kétā ā́ dabhnuvanti bhū́rṇayaḥ,ná tvā kétā ā́ dabhnuvanti bhū́rṇayaḥ,L L LL L LSLS LSL,"_LL LL ,L LS|LS LSL","[ná t, vā, ké, tā, ā́, dabh, nu, van, ti, bhū́...",12,...,scansion_post_caesura=LL,dabhnu,"[6, 7]",6,7,S,X,[6],[6],root_vowel


### Checking no of lines that have stanza meter not set

In [9]:
# total: 77 > 194
len(df_rv_lines_with_meter[COLUMNS_SELECTED].query('stanza_meter == ""'))

194